In [ ]:
from mss import mss
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from PIL import Image
#import pyautogui
from time import sleep
import winsound
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

In [ ]:
class Dataset(torch.utils.data.Dataset):
    '''
    Generates commands for Reader, a multi-label classifier
    (do not confuse with multi-class classification)
    '''
    
    def __init__(
        self,
        data=None,
        labels=None,
        top=0,
        left=0,
        width=1920,
        height=1080,
        resize=None
    ):

        # Window resolutions for the screen grabber
        self.top = top
        self.left = left
        self.width = width
        self.height = height

        self.resize = resize # For reducing the images. Must be a tuple (Height, Width)

        self.data = data # The inputs

        self.labels = labels # The list of possible outputs, where each item is a character


    # Pytorch's Dataset functions will only be used in Studying mode
    def __getitem__(self, idx):

        inputs = self.data[idx]
        labels = self.labels[idx]

        return inputs, labels


    def __len__(self):

        return len(self.data)

    def record_screen_region(self, number_of_screenshots, screenshot_delay, grayscale=False, resize=False, path=None):
        '''
        To capture screen regions, which will serve as inputs to the model
        '''

        print(f"Ok. Screenshot capture will begin in 5 seconds")

        sleep(5)

        winsound.PlaySound('D:/Python/Audio/English/chiara_hacking_1_en.wav', winsound.SND_FILENAME) # Just to know if everything's ok

        for i in range(number_of_screenshots):

            with mss() as sct:

                frame = sct.grab(monitor={"top": self.top, "left": self.left, "width": self.width, "height": self.height})
                frame = Image.frombytes("RGB", frame.size, frame.bgra, 'raw', 'BGRX')

            if grayscale:

                frame = frame.convert('L')

            if resize:

                frame = frame.resize(self.resize)

            frame.save(f"{path}/{i}.png")

            sleep(screenshot_delay)
        
        print("Screenshot capture finished!")

        winsound.PlaySound('D:/Python/Audio/English/chiara_hacking_1_en.wav', winsound.SND_FILENAME)

In [ ]:
characters = [str(x) for x in range(501)]
characters = characters + ['LIFE', 'SCORE', 'AURA', 'BOMBS']

In [ ]:
labels = [
    (0., 0., 0., 0.), (1., 0., 0., 0.), (1., 0., 0., 2.), (1., 0., 0., 4.), (1., 0., 0., 8.),
    (1., 0., 0., 9.), (1., 1., 0., 10.), (2., 2., 0., 12.), (2., 2., 0., 23.), (3., 6., 0., 23.),
    (3., 10., 0., 23.), (3., 11., 0., 23.), (3., 12., 0., 23.), (3., 13., 0., 23.), (3., 14., 0., 23.),
    (3., 14., 0., 23.), (3., 16., 0., 23.), (3., 17., 0., 23.), (3., 17., 0., 23.), (3., 18., 0., 23.),
    (3., 19., 0., 23.), (3., 19., 0., 23.), (3., 20., 0., 23.), (3., 21., 0., 23.), (3., 21., 0., 23.),
    (3., 22., 0., 23.), (3., 22., 0., 23.), (3., 22., 0., 23.), (3., 22., 0., 23.), (3., 22., 0., 23.),
    (3., 22., 0., 23.), (3., 22., 0., 23.), (3., 22., 0., 23.), (3., 22., 0., 24.), (4., 22., 0., 27.),
    (4., 22., 0., 27.), (4., 22., 0., 34.), (4., 22., 0., 53.), (4., 22., 0., 56.), (5., 22., 0., 61.),
    (5., 22., 0., 65.), (5., 22., 0., 71.), (5., 22., 0., 76.), (6., 22., 0., 100.), (6., 22., 0., 115.),
    (6., 22., 0., 132.), (6., 22., 0., 148.), (6., 22., 0., 151.), (7., 23., 0., 172.), (7., 23., 0., 182.),
    (7., 23., 0., 190.), (8., 23., 0., 204.), (9., 23., 0., 221.), (10., 23., 0., 233.), (12., 23., 0., 234.)
]

In [ ]:
dataset = Dataset(data=None, labels=labels, top=1, left=1632, width=1823-1632, height=30-1)

In [ ]:
dataset.record_screen_region(2000, 1, path='D:/Python/Projects/Hakisa/Preprocessing/Reader_BH')

In [ ]:
import os

images_by_order = []

for directory, _, files in os.walk("D:/Python/Projects/Hakisa/Preprocessing/Reader_BH"):

    for file in files:

        file = file.split('.')
        file = file[0] # Getting exclusively the number

        images_by_order.append(file)

images_by_order = sorted([int(x) for x in images_by_order])

In [ ]:
# Since many images are equal to each other, we'll select a few.

images = [
    0, 4, 7, 10, 18, 19, 32, 56, 75, 95, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
    121, 122, 123, 124, 125, 126, 127, 128, 129, 142, 159, 164, 173, 174, 184, 187, 217, 237, 242, 249, 252, 267, 295, 304, 334,
    374, 396, 408
]

In [ ]:
images = [i for i in images_by_order if i in images]

In [ ]:
images_data = []

for i in images:

    i = directory + '/' + str(i) + '.png'
    image = Image.open(i)
    array = np.array(image, dtype=np.float32)
    image.close()
    array = array/255 # Note that the data must be within [0, 1] for matplotlib.
    images_data.append(array)

images_data = np.stack(images_data, 0)

In [ ]:
images_data = torch.from_numpy(images_data)
images_data = images_data.view(images_data.size(0), images_data.size(3), images_data.size(1), images_data.size(2))
print(images_data.size())

In [ ]:
dataset.data = images_data

In [ ]:
class OCR(torch.nn.Module):
    '''
    Simple model for Optical Character Recognition

    Use it on your desired game specifically to help with the reward function.
    Be careful with overfitting.
    '''

    def __init__(self, n_channels, height, width, n_characters):

        super(OCR, self).__init__()

        # Input size: 3x191x29 = 16,617

        self.neuron1 = torch.nn.Linear(n_channels*height*width, 1000)
        self.neuron2 = torch.nn.Linear(1000, 1000)
        self.neuron3 = torch.nn.Linear(1000, 1000)

        # Remove or add neurons according to how many strings you wish to extract

        self.neuron_stringA = torch.nn.Linear(1000, n_characters)
        self.neuron_stringB = torch.nn.Linear(1000, n_characters)
        self.neuron_stringC = torch.nn.Linear(1000, n_characters)
        self.neuron_stringD = torch.nn.Linear(1000, n_characters)

        self.dropout = torch.nn.Dropout(0.5)
        self.PRelu = torch.nn.PReLU()
        

    def forward(self, input):

        input = input.view(input.size(0), -1)

        x = self.neuron1(input)
        x = self.dropout(x)
        x = self.PRelu(x)

        x = self.neuron2(x)
        x = self.dropout(x)
        x = self.PRelu(x)

        x = self.neuron3(x)
        x = self.dropout(x)
        x = self.PRelu(x)

        x = x.view(x.size(0), -1)

        stringA = self.neuron_stringA(x)
        stringB = self.neuron_stringB(x)
        stringC = self.neuron_stringC(x)
        stringD = self.neuron_stringD(x)

        output = (stringA, stringB, stringC, stringD)

        return output

In [ ]:
reader = OCR(3, 191, 29, len(characters)).to(device)

In [ ]:
from torchsummary import summary

summary(reader, (3, 191, 29))

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.Adam(reader.parameters(), lr=1e-4, eps=1e-8)

criterion = torch.nn.CrossEntropyLoss()

EPOCHS = 10 # 100 epochs at most.

In [ ]:
for epoch in range(EPOCHS):

    epoch_loss = 0.

    for i, (frame, labels) in enumerate(dataloader):

        reader.zero_grad()

        frame = frame.to(device).double()

        output = reader(frame)

        kills_loss = criterion(output[0], labels[0].to(device).long())
        deaths_loss = criterion(output[1], labels[1].to(device).long())
        assists_loss = criterion(output[2], labels[2].to(device).long())
        farm_loss = criterion(output[3], labels[3].to(device).long())

        loss = kills_loss + deaths_loss + assists_loss + farm_loss

        loss.backward()

        epoch_loss += loss.item()

        grads = torch.mean(reader.neuron1.weight.grad)

        optimizer.step()

    if epoch % (EPOCHS*0.1) == 0:

        print(f"{epoch+1}/{EPOCHS}")
        print(f"Current Loss: {epoch_loss}\tGradients Average: {grads}")

In [ ]:
teste = frame.view(frame.size(0), frame.size(2), frame.size(3), frame.size(1))
teste = teste[0].cpu().numpy()

plt.imshow(teste)
plt.show()

print(output[0][0].argmax())